In [1]:
import numpy as np
import h5py
import pandas as pd

In [2]:
f = h5py.File('Data/train.h5','r')

In [3]:
def printName(name):
    print(name)

In [4]:
f.visit(printName)

train
train/axis0
train/axis1
train/block0_items
train/block0_values
train/block1_items
train/block1_values


In [5]:
axis0 = f["train/axis0"][:]
axis1 = f["train/axis1"][:]
block0_items = f["train/block0_items"][:]
block0_values = f["train/block0_values"][:]
block1_items = f["train/block1_items"][:]
block1_values = f["train/block1_values"][:]

In [6]:
axis0.shape

(111,)

In [7]:
axis1.shape

(1710756,)

In [8]:
block0_items.shape

(2,)

In [9]:
block0_values.shape

(1710756, 2)

In [10]:
block1_items.shape

(109,)

In [11]:
block1_values.shape

(1710756, 109)

In [12]:
data = np.hstack((block0_values,block1_values))
columns = axis0
index = axis1

In [13]:
DF = pd.DataFrame(data=data,index=index,columns=columns)

In [14]:
DF.head(block1_values.shape[0]).isnull().sum()

id                     0
timestamp              0
derived_0          72959
derived_1          81029
derived_2         398651
derived_3         149471
derived_4         406458
fundamental_0      23947
fundamental_1     679070
fundamental_2     368840
fundamental_3     454380
fundamental_5     962020
fundamental_6     701625
fundamental_7      26340
fundamental_8     373166
fundamental_9     565567
fundamental_10    112977
fundamental_11    368840
fundamental_12    110871
fundamental_13    355138
fundamental_14    356084
fundamental_15    354897
fundamental_16    355138
fundamental_17     97222
fundamental_18     15833
fundamental_19     54588
fundamental_20    110871
fundamental_21     54333
fundamental_22    558488
fundamental_23    356723
                   ...  
technical_13        4764
technical_14       14184
technical_16       19981
technical_17        4279
technical_18       20016
technical_19        2320
technical_20        4764
technical_21        2236
technical_22           0


In [15]:
dff  = DF[(DF.T.isnull()).any()]

In [16]:
dff.head()

,id,timestamp,derived_0,derived_1,derived_2,derived_3,derived_4,fundamental_0,fundamental_1,fundamental_2,...,technical_36,technical_37,technical_38,technical_39,technical_40,technical_41,technical_42,technical_43,technical_44,y
0,10.0,0.0,0.370326,-0.006316,0.222831,-0.213030,0.729277,-0.335633,0.113292,1.621238,...,0.775208,NaN,NaN,NaN,-0.414776,NaN,NaN,-2.0,NaN,-0.011753
1,11.0,0.0,0.014765,-0.038064,-0.017425,0.320652,-0.034134,0.004413,0.114285,-0.210185,...,0.025590,NaN,NaN,NaN,-0.273607,NaN,NaN,-2.0,NaN,-0.001240
2,12.0,0.0,-0.010622,-0.050577,3.379575,-0.157525,-0.068550,-0.155937,1.219439,-0.764516,...,0.151881,NaN,NaN,NaN,-0.175710,NaN,NaN,-2.0,NaN,-0.020940
3,25.0,0.0,NaN,NaN,NaN,NaN,NaN,0.178495,NaN,-0.007262,...,1.035936,NaN,NaN,NaN,-0.211506,NaN,NaN,-2.0,NaN,-0.015959
4,26.0,0.0,0.176693,-0.025284,-0.057680,0.015100,0.180894,0.139445,-0.125687,-0.018707,...,0.630232,NaN,NaN,NaN,-0.001957,NaN,NaN,0.0,NaN,-0.007338


In [17]:
df = DF

In [18]:
df_idx = df.iloc[:,:].isnull()
no_nan = np.sum(np.sum(df_idx,axis=1) > 0) # number of rows with at least 1 NAN
no_nnan = len(df_idx) - no_nan

print 'Number of rows with a NaN: {0}'.format(no_nan)
print 'Number of rows without a NaN: {0}'.format(no_nnan)

Number of rows with a NaN: 1487716
Number of rows without a NaN: 223040


In [19]:
df_nnan = df.dropna(axis=0,how='any')

In [20]:
df_nnan.shape

(223040, 111)

In [21]:
import numpy as np
data = df_nnan[:3000]
data = np.array(data)
data.shape
X = data[:,2:-1]
Y = data[:,-1].reshape(3000,1)
# data

In [22]:
X.shape

(3000, 108)

In [23]:
Y.shape

(3000, 1)

In [25]:
from sklearn.cross_validation import train_test_split
trainX,testx,trainY,testy = train_test_split(X,Y,test_size = 0.1)

/Users/Steffi/tensorflow/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [26]:
trainX.shape

(2700, 108)

In [27]:
# Import Keras Libraries
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import SGD
import sklearn

Using TensorFlow backend.


In [28]:
# Define Model Parameters
nb_feat   = 108   # no. of features/columns of input
L1_units  = 100   # no. of nodes in Layer 1
L2_units  = 80   # no. of nodes in Layer 2
L3_units  = 50   # no. of nodes in Layer 2
L4_units = 10
nb_class  = 1    # no. of output classes

model = Sequential()
# Neural Network Model
model = Sequential()                             # Sequential network model description
model.add(Dense(L1_units,input_dim=108))    # Add 1st Dense Layer
model.add(Activation("relu"))                    # Add activation function

model.add(Dense(L2_units,))                       # Add 2nd Dense Layer
model.add(Activation("relu"))                    # Add activation function

model.add(Dense(L3_units))                       # Add 3nd Dense Layer
model.add(Activation("relu"))                    # Add activation function

model.add(Dense(L4_units))                       # Add 4nd Dense Layer
model.add(Activation("relu"))                    # Add activation function

model.add(Dense(nb_class))                       # Add 5rd Dense Layer, also the classification layer
model.add(Activation('softmax'))                 # Add sigmoid classification


In [29]:
model.compile(loss='mean_squared_error',optimizer='rmsprop', metrics=['accuracy'])

In [30]:
log = model.fit(trainX, trainY,nb_epoch=30, batch_size=20, verbose=2,validation_data=(testx, testy))
# df3k = df_3k.iloc[:,2:]
# sname = 'xtrip.csv'
# np.savetxt(sname,delimeter = ',')

Train on 2700 samples, validate on 300 samples
Epoch 1/30
1s - loss: 0.9989 - acc: 0.0000e+00 - val_loss: 0.9965 - val_acc: 0.0000e+00
Epoch 2/30
0s - loss: 0.9989 - acc: 0.0000e+00 - val_loss: 0.9965 - val_acc: 0.0000e+00
Epoch 3/30
0s - loss: 0.9989 - acc: 0.0000e+00 - val_loss: 0.9965 - val_acc: 0.0000e+00
Epoch 4/30
0s - loss: 0.9989 - acc: 0.0000e+00 - val_loss: 0.9965 - val_acc: 0.0000e+00
Epoch 5/30
0s - loss: 0.9989 - acc: 0.0000e+00 - val_loss: 0.9965 - val_acc: 0.0000e+00
Epoch 6/30
0s - loss: 0.9989 - acc: 0.0000e+00 - val_loss: 0.9965 - val_acc: 0.0000e+00
Epoch 7/30
0s - loss: 0.9989 - acc: 0.0000e+00 - val_loss: 0.9965 - val_acc: 0.0000e+00
Epoch 8/30
0s - loss: 0.9989 - acc: 0.0000e+00 - val_loss: 0.9965 - val_acc: 0.0000e+00
Epoch 9/30
0s - loss: 0.9989 - acc: 0.0000e+00 - val_loss: 0.9965 - val_acc: 0.0000e+00
Epoch 10/30
0s - loss: 0.9989 - acc: 0.0000e+00 - val_loss: 0.9965 - val_acc: 0.0000e+00
Epoch 11/30
0s - loss: 0.9989 - acc: 0.0000e+00 - val_loss: 0.9965 - val